# Sentiment Infomation

Display sentimental polarity pie chart and trend chart 

# Load Data

In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('./now_news_preprocessed.csv',sep='|')

In [5]:
df.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,column/_20220505_1001,2022-05-05,焦點,配發率32%！富邦金每股配4元股利、現金股利歷史新高,富邦金控今（5）日公告，董事會決議通過擬配發普通股每股現金股利3.5元及股票股利0.5元，合...,0.0,"['富邦金這次同時配發股票股利每股0.5元', '包含富邦人壽上繳155.78億元、台北富邦...","[('股利', 18), ('富邦', 12), ('現金', 9), ('股票', 5),...","['富邦', '金控', '今', '（5', '）', '日', '公告', '，', '...","['富邦', '金控', '董事會', '通過', '普通股', '現金', '股利', '...","[NerToken(word='富邦金控', ner='ORG', idx=(0, 4)),...","[('富邦', 'Nb'), ('金控', 'Na'), ('今', 'Nd'), ('（5...",https://www.nownews.com/news/5796259,https://media.nownews.com/nn_media/thumbnail/2...


In [6]:
df.shape

(3671, 14)

# Step 1: Filter news articles using the following function

In [7]:
from datetime import datetime, timedelta

In [8]:
from datetime import datetime, timedelta
# Searching keywords from "content" column or tokens_v2
def filter_df_via_tokens(query_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date_delta = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')
    start_date_min = df.date.min()
    # set start_date as the larger one from the start_date_delta and start_date_min 開始時間選資料最早時間與周數:兩者較晚者
    start_date = max(start_date_delta,   start_date_min)


    # (1) proceed filtering: a duration of a period of time
    # 期間條件
    period_condition = (df.date >= start_date) & (df.date <= end_date) 
    
    # (2) proceed filtering: news category
    # 新聞類別條件
    if (cate == "全部"):
        condition = period_condition  # "全部"類別不必過濾新聞種類
    else:
        # 過濾category新聞類別條件
        condition = period_condition & (df.category == cate)

    # (3) proceed filtering: and or
    # and or 條件
    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = condition & df.tokens_v2.apply(lambda text: all((qk in text) for qk in query_keywords)) #寫法:all()
    elif (cond == 'or'):
        # query keywords condition使用者輸入關鍵字條件
        condition = condition & df.tokens_v2.apply(lambda text: any((qk in text) for qk in query_keywords)) #寫法:any()
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    return df_query


In [9]:
from datetime import datetime, timedelta
# Searching keywords from "content" column
# Here this function uses df.content column, while filter_dataFrame() uses df.tokens_v2
def filter_df_via_content(query_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date_delta = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')
    start_date_min = df.date.min()
    # set start_date as the larger one from the start_date_delta and start_date_min 開始時間選資料最早時間與周數:兩者較晚者
    start_date = max(start_date_delta,   start_date_min)


    # (1) proceed filtering: a duration of a period of time
    # 期間條件
    period_condition = (df.date >= start_date) & (df.date <= end_date) 
    
    # (2) proceed filtering: news category
    # 新聞類別條件
    if (cate == "全部"):
        condition = period_condition  # "全部"類別不必過濾新聞種類
    else:
        # 過濾category新聞類別條件
        condition = period_condition & (df.category == cate)

    # (3) proceed filtering: and or
    # and or 條件
    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = condition & df.content.apply(lambda text: all((qk in text) for qk in query_keywords)) #寫法:all()
    elif (cond == 'or'):
        # query keywords condition使用者輸入關鍵字條件
        condition = condition & df.content.apply(lambda text: any((qk in text) for qk in query_keywords)) #寫法:any()
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    return df_query


# Step 2: Sentimental polarity score計算整體情緒分數(影響力)

Count number of article with Positive, Negative, and Neutral polarity

In [10]:
# sentimental polarity score
def get_article_sentiment(df_query):
    # df_query = df[df['tokens'].str.contains(query_key)]
    sentiCount = {'pos': 0, 'neg': 0, 'obj': 0}
    sentiPercnt = {'pos': 0, 'neg': 0, 'obj': 0}
    numberOfArticle = len(df_query)
    for senti in df_query.sentiment:
        # 判斷文章的情緒極性
        if senti >= 0.75:
            sentiCount['pos'] += 1
        elif senti <= 0.4:
            sentiCount['neg'] += 1
        else:
            sentiCount['obj'] += 1
    for polar in sentiCount :
        try:
            sentiPercnt[polar]=int(sentiCount[polar]/numberOfArticle*100)
            # sentiPercnt[polar]=round(sentiCount[polar]/numberOfArticle,2) # 0.75
        except:
            sentiPercnt[polar] = 0 # 若分母 numberOfArticle=0會報錯
    return sentiCount, sentiPercnt


In [11]:
query_keywords = ['黃珊珊']
cond='or'
weeks=12
cate='全部'
df_query = filter_df_via_content(query_keywords, cond, cate, weeks)
sentiCount, sentiPercnt = get_article_sentiment(df_query)

In [12]:
sentiCount

{'pos': 1, 'neg': 48, 'obj': 0}

In [13]:
sentiPercnt

{'pos': 2, 'neg': 97, 'obj': 0}

In [14]:
'正向:{}%, 中立:{}%, 負向:{}%'.format(str(sentiPercnt['pos']), str(sentiPercnt['obj']),str(sentiPercnt['neg']))


'正向:2%, 中立:0%, 負向:97%'

In [15]:
[sentiPercnt[p] for p in ['pos', 'obj', 'neg']]

[2, 0, 97]

# Step 3: Count positive and negative articles and draw line chart

The number of articles for positive and negative sentimental polarity.

This is the data for drawing daily-basis line chart on a page.

    The data format for the line chart is as follows.

    [{'x': '2020-03-05', 'y': 1},
    {'x': '2020-03-06', 'y': 0},
    {'x': '2020-03-07', 'y': 0},
    {'x': '2020-03-08', 'y': 0},
    {'x': '2020-03-09', 'y': 0},
    {'x': '2020-03-10', 'y': 5},
    {'x': '2020-03-11', 'y': 5},
    {'x': '2020-03-12', 'y': 7},
    {'x': '2020-03-13', 'y': 62},
    {'x': '2020-03-14', 'y': 29}]

In [16]:
def get_daily_basis_sentiment_count(df_query, sentiment_type='pos', freq_type='D'):

    # 自訂正負向中立的標準，sentiment score是機率值，介於0~1之間
    # Using lambda to determine if an article is postive or not.
    if sentiment_type == 'pos':
        lambda_function = lambda senti: 1 if senti >= 0.6 else 0 #大於0.6為正向 
    elif sentiment_type == 'neg':
        lambda_function = lambda senti: 1 if senti <= 0.4 else 0 #小於0.4為負向
    elif sentiment_type == 'neutral':
        lambda_function = lambda senti: 1 if senti > 0.4 & senti < 0.4 else 0 #介於中間為中立
    else:
        return None 
        
    freq_df = pd.DataFrame({'date_index': pd.to_datetime(df_query.date),
                             'frequency': [lambda_function(senti) for senti in df_query.sentiment]})
    # Group rows by the date to the daily number of articles. 加總合併同一天新聞，篇數就被計算好了
    freq_df_group = freq_df.groupby(pd.Grouper(key='date_index',freq=freq_type)).sum()
    
    # 'date_index'為index(索引)，將其變成欄位名稱，inplace=True表示原始檔案會被異動
    freq_df_group.reset_index(inplace=True)
    
    # x,y，用於畫趨勢線圖
    xy_line_data = [{'x':date.strftime('%Y-%m-%d'),'y':freq} for date, freq in zip(freq_df_group.date_index,freq_df_group.frequency)]
    return xy_line_data

In [17]:
query_keywords = ['黃珊珊']
cond='or'
weeks=12
cate='全部'
df_query = filter_df_via_content(query_keywords, cond, cate, weeks)

In [18]:
get_daily_basis_sentiment_count(df_query, sentiment_type='pos')

[{'x': '2022-05-06', 'y': 0},
 {'x': '2022-05-07', 'y': 0},
 {'x': '2022-05-08', 'y': 0},
 {'x': '2022-05-09', 'y': 0},
 {'x': '2022-05-10', 'y': 0},
 {'x': '2022-05-11', 'y': 0},
 {'x': '2022-05-12', 'y': 0},
 {'x': '2022-05-13', 'y': 0},
 {'x': '2022-05-14', 'y': 0},
 {'x': '2022-05-15', 'y': 0},
 {'x': '2022-05-16', 'y': 0},
 {'x': '2022-05-17', 'y': 0},
 {'x': '2022-05-18', 'y': 0},
 {'x': '2022-05-19', 'y': 0},
 {'x': '2022-05-20', 'y': 0},
 {'x': '2022-05-21', 'y': 0},
 {'x': '2022-05-22', 'y': 0},
 {'x': '2022-05-23', 'y': 0},
 {'x': '2022-05-24', 'y': 0},
 {'x': '2022-05-25', 'y': 0},
 {'x': '2022-05-26', 'y': 0},
 {'x': '2022-05-27', 'y': 0},
 {'x': '2022-05-28', 'y': 0},
 {'x': '2022-05-29', 'y': 0},
 {'x': '2022-05-30', 'y': 0},
 {'x': '2022-05-31', 'y': 0},
 {'x': '2022-06-01', 'y': 0},
 {'x': '2022-06-02', 'y': 0},
 {'x': '2022-06-03', 'y': 1},
 {'x': '2022-06-04', 'y': 0},
 {'x': '2022-06-05', 'y': 0},
 {'x': '2022-06-06', 'y': 0},
 {'x': '2022-06-07', 'y': 0}]

In [19]:
get_daily_basis_sentiment_count(df_query, sentiment_type='neg')

[{'x': '2022-05-06', 'y': 1},
 {'x': '2022-05-07', 'y': 0},
 {'x': '2022-05-08', 'y': 0},
 {'x': '2022-05-09', 'y': 10},
 {'x': '2022-05-10', 'y': 6},
 {'x': '2022-05-11', 'y': 2},
 {'x': '2022-05-12', 'y': 1},
 {'x': '2022-05-13', 'y': 0},
 {'x': '2022-05-14', 'y': 0},
 {'x': '2022-05-15', 'y': 0},
 {'x': '2022-05-16', 'y': 0},
 {'x': '2022-05-17', 'y': 0},
 {'x': '2022-05-18', 'y': 0},
 {'x': '2022-05-19', 'y': 0},
 {'x': '2022-05-20', 'y': 0},
 {'x': '2022-05-21', 'y': 0},
 {'x': '2022-05-22', 'y': 0},
 {'x': '2022-05-23', 'y': 7},
 {'x': '2022-05-24', 'y': 7},
 {'x': '2022-05-25', 'y': 5},
 {'x': '2022-05-26', 'y': 0},
 {'x': '2022-05-27', 'y': 0},
 {'x': '2022-05-28', 'y': 0},
 {'x': '2022-05-29', 'y': 0},
 {'x': '2022-05-30', 'y': 2},
 {'x': '2022-05-31', 'y': 1},
 {'x': '2022-06-01', 'y': 0},
 {'x': '2022-06-02', 'y': 2},
 {'x': '2022-06-03', 'y': 1},
 {'x': '2022-06-04', 'y': 0},
 {'x': '2022-06-05', 'y': 0},
 {'x': '2022-06-06', 'y': 0},
 {'x': '2022-06-07', 'y': 3}]

## How does this work?

In [20]:
# This is what we used to calulate daily freqncy of keyword in our previous app
pd.DataFrame({'date_index':pd.to_datetime( df_query.date ),'frequency':[1 for _ in range(len(df_query))]})

,date_index,frequency
58,2022-05-06,1
212,2022-06-07,1
218,2022-06-07,1
221,2022-06-07,1
609,2022-05-09,1
615,2022-05-09,1
619,2022-05-09,1
662,2022-05-09,1
672,2022-05-09,1
680,2022-05-09,1


In [21]:
# Now we need to modify the above line as follows.
# Using lambda to determine if an article is postive or not.
pd.DataFrame({'date_index':pd.to_datetime( df_query.date ),'pos':[ (lambda senti: 1 if senti >= 0.7 else 0)(senti) for senti in df_query.sentiment]})

,date_index,pos
58,2022-05-06,0
212,2022-06-07,0
218,2022-06-07,0
221,2022-06-07,0
609,2022-05-09,0
615,2022-05-09,0
619,2022-05-09,0
662,2022-05-09,0
672,2022-05-09,0
680,2022-05-09,0


In [22]:
# What is the following lambda function? It is a little bite hard to understand.
[ (lambda senti: 1 if senti >= 0.7 else 0)(senti) for senti in df_query.sentiment]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1]

In [23]:
lambda x: 1 if x >= 0.7 else 0

<function __main__.<lambda>(x)>

In [24]:
(lambda x: 1 if x >= 0.7 else 0)(0.9)

1

In [25]:
(lambda x: 1 if x >= 0.7 else 0)(0.2)

0

In [26]:
pd.to_datetime( df_query.date )

58     2022-05-06
212    2022-06-07
218    2022-06-07
221    2022-06-07
609    2022-05-09
615    2022-05-09
619    2022-05-09
662    2022-05-09
672    2022-05-09
680    2022-05-09
726    2022-05-09
736    2022-05-09
917    2022-05-09
1033   2022-05-09
1092   2022-05-10
1103   2022-05-10
1130   2022-05-10
1197   2022-05-10
1204   2022-05-10
1223   2022-05-10
1333   2022-05-11
1343   2022-05-11
1427   2022-05-12
1777   2022-05-23
1782   2022-05-23
1796   2022-05-23
1856   2022-05-23
1863   2022-05-23
1866   2022-05-23
1869   2022-05-23
1920   2022-05-24
1926   2022-05-24
1928   2022-05-24
1930   2022-05-24
1939   2022-05-24
1949   2022-05-24
2030   2022-05-24
2098   2022-05-25
2100   2022-05-25
2110   2022-05-25
2112   2022-05-25
2115   2022-05-25
2257   2022-05-30
2370   2022-05-30
2511   2022-05-31
2886   2022-06-02
2930   2022-06-02
3027   2022-06-03
3051   2022-06-03
Name: date, dtype: datetime64[ns]

In [27]:
pd.DataFrame({'date_index':pd.to_datetime( df_query.date ),'frequency':[ (lambda senti: 1 if senti >= 0.7 else 0)(senti) for senti in df_query.sentiment]})

,date_index,frequency
58,2022-05-06,0
212,2022-06-07,0
218,2022-06-07,0
221,2022-06-07,0
609,2022-05-09,0
615,2022-05-09,0
619,2022-05-09,0
662,2022-05-09,0
672,2022-05-09,0
680,2022-05-09,0


In [28]:
# a dict with 'date_index' key and 'frequency' value ==> A dict is consist of several (key, value) pairs.
{'date_index':pd.to_datetime( df_query.date ),'frequency':[ (lambda senti: 1 if senti >= 0.7 else 0)(senti) for senti in df_query.sentiment]}

{'date_index': 58     2022-05-06
 212    2022-06-07
 218    2022-06-07
 221    2022-06-07
 609    2022-05-09
 615    2022-05-09
 619    2022-05-09
 662    2022-05-09
 672    2022-05-09
 680    2022-05-09
 726    2022-05-09
 736    2022-05-09
 917    2022-05-09
 1033   2022-05-09
 1092   2022-05-10
 1103   2022-05-10
 1130   2022-05-10
 1197   2022-05-10
 1204   2022-05-10
 1223   2022-05-10
 1333   2022-05-11
 1343   2022-05-11
 1427   2022-05-12
 1777   2022-05-23
 1782   2022-05-23
 1796   2022-05-23
 1856   2022-05-23
 1863   2022-05-23
 1866   2022-05-23
 1869   2022-05-23
 1920   2022-05-24
 1926   2022-05-24
 1928   2022-05-24
 1930   2022-05-24
 1939   2022-05-24
 1949   2022-05-24
 2030   2022-05-24
 2098   2022-05-25
 2100   2022-05-25
 2110   2022-05-25
 2112   2022-05-25
 2115   2022-05-25
 2257   2022-05-30
 2370   2022-05-30
 2511   2022-05-31
 2886   2022-06-02
 2930   2022-06-02
 3027   2022-06-03
 3051   2022-06-03
 Name: date, dtype: datetime64[ns],
 'frequency': [0,
 

In [29]:
# Convert the above dict into DataFrame and assign it to query_freq
freq_df = pd.DataFrame({'date_index':pd.to_datetime( df_query.date ),'frequency':[ (lambda senti: 1 if senti >= 0.7 else 0)(senti) for senti in df_query.sentiment]})

In [30]:
# Group rows by the date to the daily number of articles. 加總合併同一天新聞，篇數就被計算好了
freq_df_group = freq_df.groupby(pd.Grouper(key='date_index',freq='D')).sum()

In [31]:
freq_df_group

,frequency
date_index,
2022-05-06,0
2022-05-07,0
2022-05-08,0
2022-05-09,0
2022-05-10,0
2022-05-11,0
2022-05-12,0
2022-05-13,0
2022-05-14,0


In [32]:
# 'date_index'為index(索引)，將其變成欄位名稱，inplace=True表示原始檔案會被異動
freq_df_group.reset_index(inplace=True)
# freq_df_group = freq_data_group.reset_index() # 這樣也可以得到同樣結果


In [33]:

# 有時間變數x,y，用於畫趨勢線圖
line_xy_data = [{'x':date.strftime('%Y-%m-%d'),'y':pos} for date, pos in zip(freq_df_group.date_index,freq_df_group.frequency)]

In [34]:
line_xy_data

[{'x': '2022-05-06', 'y': 0},
 {'x': '2022-05-07', 'y': 0},
 {'x': '2022-05-08', 'y': 0},
 {'x': '2022-05-09', 'y': 0},
 {'x': '2022-05-10', 'y': 0},
 {'x': '2022-05-11', 'y': 0},
 {'x': '2022-05-12', 'y': 0},
 {'x': '2022-05-13', 'y': 0},
 {'x': '2022-05-14', 'y': 0},
 {'x': '2022-05-15', 'y': 0},
 {'x': '2022-05-16', 'y': 0},
 {'x': '2022-05-17', 'y': 0},
 {'x': '2022-05-18', 'y': 0},
 {'x': '2022-05-19', 'y': 0},
 {'x': '2022-05-20', 'y': 0},
 {'x': '2022-05-21', 'y': 0},
 {'x': '2022-05-22', 'y': 0},
 {'x': '2022-05-23', 'y': 0},
 {'x': '2022-05-24', 'y': 0},
 {'x': '2022-05-25', 'y': 0},
 {'x': '2022-05-26', 'y': 0},
 {'x': '2022-05-27', 'y': 0},
 {'x': '2022-05-28', 'y': 0},
 {'x': '2022-05-29', 'y': 0},
 {'x': '2022-05-30', 'y': 0},
 {'x': '2022-05-31', 'y': 0},
 {'x': '2022-06-01', 'y': 0},
 {'x': '2022-06-02', 'y': 0},
 {'x': '2022-06-03', 'y': 1},
 {'x': '2022-06-04', 'y': 0},
 {'x': '2022-06-05', 'y': 0},
 {'x': '2022-06-06', 'y': 0},
 {'x': '2022-06-07', 'y': 0}]

# Step 4: Prepare for response data 

In [36]:
def prepare_for_response(query_keywords, cond, cate, weeks):

    # Proceed filtering
    df_query = filter_df_via_content(query_keywords, cond, cate, weeks)
    
    sentiCount, sentiPercnt = get_article_sentiment(df_query)

    if weeks <= 4:
        freq_type = 'D'
    else:
        freq_type = 'W'

    line_data_pos = get_daily_basis_sentiment_count(df_query, sentiment_type='pos', freq_type=freq_type)
    line_data_neg = get_daily_basis_sentiment_count(df_query, sentiment_type='neg', freq_type=freq_type)

    response = {
        'sentiCount': sentiCount,
        'data_pos':line_data_pos,
        'data_neg':line_data_neg,
    }
    return response

In [37]:
query_keywords = ['黃珊珊']
cond='or'
weeks=12
cate='全部'
prepare_for_response(query_keywords, cond, cate, weeks)

{'sentiCount': {'pos': 1, 'neg': 48, 'obj': 0},
 'data_pos': [{'x': '2022-05-08', 'y': 0},
  {'x': '2022-05-15', 'y': 0},
  {'x': '2022-05-22', 'y': 0},
  {'x': '2022-05-29', 'y': 0},
  {'x': '2022-06-05', 'y': 1},
  {'x': '2022-06-12', 'y': 0}],
 'data_neg': [{'x': '2022-05-08', 'y': 1},
  {'x': '2022-05-15', 'y': 19},
  {'x': '2022-05-22', 'y': 0},
  {'x': '2022-05-29', 'y': 19},
  {'x': '2022-06-05', 'y': 6},
  {'x': '2022-06-12', 'y': 3}]}

# Step 5: Django views.py
    (1)app name: app_userkey_sentiment
    (2)namespace defined in urls.py: 
        app_name="namespace_userkey_sentiment"  
        or app_name="app_userkey_sentiment"
    (3) home.html


In [34]:
from django.http import JsonResponse
from django.shortcuts import render
import pandas as pd
from datetime import datetime, timedelta
from django.views.decorators.csrf import csrf_exempt


# (1) Load news data--approach 1 直接指定某個csv檔案
def load_df_data_v1():
    global df # global variable
    # df = pd.read_csv('app_user_keyword/dataset/cna_news_200_preprocessed.csv',sep='|')
    df = pd.read_csv('app_user_keyword_sentiment/dataset/news_dataset_preprocessed_for_django.csv',sep='|')

# (2) Load news data--approach 2 跟隔壁的app借用df
# import from app_user_keyword.views and use df later
import app_user_keyword.views as userkeyword_views
def load_df_data():
    # import and use df from app_user_keyword 
    global df # global variable
    df = userkeyword_views.df

# call load data function when starting server
load_df_data_v1()
# load_df_data()

def home(request):
    return render(request, 'app_user_keyword_sentiment/home.html')

# GET: csrf_exempt is not necessary
# POST: csrf_exempt should be used
@csrf_exempt
def api_get_userkey_sentiment(request):

    userkey = request.POST['userkey']
    cate = request.POST['cate']
    cond = request.POST['cond']
    weeks = int(request.POST['weeks'])
    print(weeks)

    query_keywords = userkey.split()
    response = prepare_for_response(query_keywords, cond, cate, weeks)
  
    return JsonResponse(response)

def prepare_for_response(query_keywords, cond, cate, weeks):

    # Proceed filtering
    df_query = filter_df_via_content(query_keywords, cond, cate, weeks)
    
    sentiCount, sentiPercnt = get_article_sentiment(df_query)

    if weeks <= 4:
        freq_type = 'D'
    else:
        freq_type = 'W'

    line_data_pos = get_daily_basis_sentiment_count(df_query, sentiment_type='pos', freq_type=freq_type)
    line_data_neg = get_daily_basis_sentiment_count(df_query, sentiment_type='neg', freq_type=freq_type)

    response = {
        'sentiCount': sentiCount,
        'data_pos':line_data_pos,
        'data_neg':line_data_neg,
    }
    return response

def get_article_sentiment(df_query):
    sentiCount = {'Positive': 0, 'Negative': 0, 'Neutral': 0}
    sentiPercnt = {'Positive': 0, 'Negative': 0, 'Neutral': 0}
    numberOfArticle = len(df_query)
    for senti in df_query.sentiment:
        # determine sentimental polarity
        if float(senti) >= 0.75:
            sentiCount['Positive'] += 1
        elif float(senti) <= 0.4:
            sentiCount['Negative'] += 1
        else:
            sentiCount['Neutral'] += 1
    for polar in sentiCount :
        try:
            sentiPercnt[polar]=int(sentiCount[polar]/numberOfArticle*100)
        except:
            sentiPercnt[polar] = 0 # 若分母 numberOfArticle=0會報錯
    return sentiCount, sentiPercnt


def get_daily_basis_sentiment_count(df_query, sentiment_type='pos', freq_type='D'):

    # 自訂正負向中立的標準，sentiment score是機率值，介於0~1之間
    # Using lambda to determine if an article is postive or not.
    if sentiment_type == 'pos':
        lambda_function = lambda senti: 1 if senti >= 0.6 else 0 #大於0.6為正向 
    elif sentiment_type == 'neg':
        lambda_function = lambda senti: 1 if senti <= 0.4 else 0 #小於0.4為負向
    elif sentiment_type == 'neutral':
        lambda_function = lambda senti: 1 if senti > 0.4 & senti < 0.4 else 0 #介於中間為中立
    else:
        return None 
        
    freq_df = pd.DataFrame({'date_index': pd.to_datetime(df_query.date),
                             'frequency': [lambda_function(senti) for senti in df_query.sentiment]})
    # Group rows by the date to the daily number of articles. 加總合併同一天新聞，篇數就被計算好了
    freq_df_group = freq_df.groupby(pd.Grouper(key='date_index',freq=freq_type)).sum()
    
    # 'date_index'為index(索引)，將其變成欄位名稱，inplace=True表示原始檔案會被異動
    freq_df_group.reset_index(inplace=True)
    
    # x,y，用於畫趨勢線圖
    xy_line_data = [{'x':date.strftime('%Y-%m-%d'),'y':freq} for date, freq in zip(freq_df_group.date_index,freq_df_group.frequency)]
    return xy_line_data

# Searching keywords from "content" column
# Here this function uses df.content column, while filter_dataFrame() uses df.tokens_v2
def filter_df_via_content(query_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date_delta = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')
    start_date_min = df.date.min()
    # set start_date as the larger one from the start_date_delta and start_date_min 開始時間選資料最早時間與周數:兩者較晚者
    start_date = max(start_date_delta,   start_date_min)


    # (1) proceed filtering: a duration of a period of time
    # 期間條件
    period_condition = (df.date >= start_date) & (df.date <= end_date) 
    
    # (2) proceed filtering: news category
    # 新聞類別條件
    if (cate == "全部"):
        condition = period_condition  # "全部"類別不必過濾新聞種類
    else:
        # 過濾category新聞類別條件
        condition = period_condition & (df.category == cate)

    # (3) proceed filtering: and or
    # and or 條件
    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = condition & df.content.apply(lambda text: all((qk in text) for qk in query_keywords)) #寫法:all()
    elif (cond == 'or'):
        # query keywords condition使用者輸入關鍵字條件
        condition = condition & df.content.apply(lambda text: any((qk in text) for qk in query_keywords)) #寫法:any()
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    return df_query


print("app_userkey_sentiment was loaded!")




ModuleNotFoundError: No module named 'app_user_keyword'